d
# Delta Lake

1. Create a Delta Table
1. Understand the Transaction Log
1. Read data from your Delta Table
1. Update data in your Delta Table
1. Access previous versions of table using time travel
1. Vacuum

##### Documentation
- <a href="https://docs.delta.io/latest/quick-start.html#create-a-table" target="_blank">Delta Table</a> 
- <a href="https://databricks.com/blog/2019/08/21/diving-into-delta-lake-unpacking-the-transaction-log.html" target="_blank">Transaction Log</a> 
- <a href="https://databricks.com/blog/2019/02/04/introducing-delta-time-travel-for-large-scale-data-lakes.html" target="_blank">Time Travel</a>

In [0]:
%run "./Includes/Classroom-Setup"

Datasets mounted and student environment set up

### ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Create a Delta Table
Let's use the BedBricks events dataset

In [0]:
eventsDF = spark.read.parquet(eventsPath)
display(eventsDF)

device,ecommerce,event_name,event_previous_timestamp,event_timestamp,geo,items,traffic_source,user_first_touch_timestamp,user_id
macOS,"List(null, null, null)",warranty,1593878899217692,1593878946592107,"List(Montrose, MI)",List(),google,1593878899217692,UA000000107379500
Windows,"List(null, null, null)",press,1593876662175340,1593877011756535,"List(Northampton, MA)",List(),google,1593876662175340,UA000000107359357
macOS,"List(null, null, null)",add_item,1593878792892652,1593878815459100,"List(Salinas, CA)","List(List(null, M_STAN_T, Standard Twin Mattress, 595.0, 595.0, 1))",youtube,1593878455472030,UA000000107375547
iOS,"List(null, null, null)",mattresses,1593878178791663,1593878809276923,"List(Everett, MA)",List(),facebook,1593877903116176,UA000000107370581
Windows,"List(null, null, null)",mattresses,null,1593878628143633,"List(Cottage Grove, MN)",List(),google,1593878628143633,UA000000107377108
Windows,"List(null, null, null)",main,null,1593878634344194,"List(Medina, MN)",List(),youtube,1593878634344194,UA000000107377161
iOS,"List(null, null, null)",main,null,1593877936171803,"List(Mount Pleasant, UT)",List(),direct,1593877936171803,UA000000107370851
macOS,"List(null, null, null)",main,null,1593876843215329,"List(Piedmont, AL)",List(),instagram,1593876843215329,UA000000107360961
Android,"List(null, null, null)",warranty,1593878529774474,1593879213196400,"List(Rancho Santa Margarita, CA)",List(),instagram,1593878529774474,UA000000107376205
Windows,"List(null, null, null)",main,null,1593876713246514,"List(Elyria, OH)",List(),facebook,1593876713246514,UA000000107359805


Convert data to a Delta table using the schema provided by the DataFrame

In [0]:
deltaPath = workingDir + "/delta-events"
eventsDF.write.format("delta").mode("overwrite").save(deltaPath)

We can also create a Delta table in the metastore

In [0]:
eventsDF.write.format("delta").mode("overwrite").saveAsTable("delta_events")

Delta supports partitioning your data using unique values in a specified column.

Let's partition by state. This gives us a point of quick comparison between different parts of the US.

In [0]:
from pyspark.sql.functions import col
stateEventsDF = eventsDF.withColumn("state", col("geo.state"))

stateEventsDF.write.format("delta").mode("overwrite").partitionBy("state").option("overwriteSchema", "true").save(deltaPath)

### ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Understand the Transaction Log
We can see how Delta stores the different state partitions in separate files.

Additionally, we can also see a directory called `_delta_log`, its transaction log.

When a Delta Lake table is created, its transaction log is automatically created in the `_delta_log` subdirectory.

In [0]:
display(dbutils.fs.ls(deltaPath))

path,name,size
dbfs:/user/silvvavaleria@gmail.com/spark_programming/p/delta-events/_delta_log/,_delta_log/,0
dbfs:/user/silvvavaleria@gmail.com/spark_programming/p/delta-events/part-00000-0c8affaf-83b8-4276-abd9-3b42d0e76aa8-c000.snappy.parquet,part-00000-0c8affaf-83b8-4276-abd9-3b42d0e76aa8-c000.snappy.parquet,74600524
dbfs:/user/silvvavaleria@gmail.com/spark_programming/p/delta-events/part-00001-2995e88a-2bdb-472b-ad4d-b7d0b567f10b-c000.snappy.parquet,part-00001-2995e88a-2bdb-472b-ad4d-b7d0b567f10b-c000.snappy.parquet,74596237
dbfs:/user/silvvavaleria@gmail.com/spark_programming/p/delta-events/part-00002-744ec8e9-9770-463f-88cc-2dfaa658fdcf-c000.snappy.parquet,part-00002-744ec8e9-9770-463f-88cc-2dfaa658fdcf-c000.snappy.parquet,74575576
dbfs:/user/silvvavaleria@gmail.com/spark_programming/p/delta-events/part-00003-813f468c-7f83-44ec-9b9d-a218ed338b66-c000.snappy.parquet,part-00003-813f468c-7f83-44ec-9b9d-a218ed338b66-c000.snappy.parquet,74575459
dbfs:/user/silvvavaleria@gmail.com/spark_programming/p/delta-events/state=AK/,state=AK/,0
dbfs:/user/silvvavaleria@gmail.com/spark_programming/p/delta-events/state=AL/,state=AL/,0
dbfs:/user/silvvavaleria@gmail.com/spark_programming/p/delta-events/state=AR/,state=AR/,0
dbfs:/user/silvvavaleria@gmail.com/spark_programming/p/delta-events/state=AZ/,state=AZ/,0
dbfs:/user/silvvavaleria@gmail.com/spark_programming/p/delta-events/state=CA/,state=CA/,0


-sandbox
When changes are made to that table, these changes are recorded as ordered, atomic commits in the transaction log.

Each commit is written out as a JSON file, starting with 000000.json.

Additional changes to the table generate subsequent JSON files in ascending numerical order.

<div style="img align: center; line-height: 0; padding-top: 9px;">
  <img src="https://user-images.githubusercontent.com/20408077/87174138-609fe600-c29c-11ea-90cc-84df0c1357f1.png" width="500"/>
</div>

In [0]:
display(dbutils.fs.ls(deltaPath + "/_delta_log/"))

Next, let's take a look at a Transaction Log File.


The <a href="https://docs.databricks.com/delta/delta-utility.html" target="_blank">four columns</a> each represent a different part of the very first commit to the Delta Table, creating the table.
- The `add` column has statistics about the DataFrame as a whole and individual columns.
- The `commitInfo` column has useful information about what the operation was (WRITE or READ) and who executed the operation.
- The `metaData` column contains information about the column schema.
- The `protocol` version contains information about the minimum Delta version necessary to either write or read to this Delta Table.

In [0]:
display(spark.read.json(deltaPath + "/_delta_log/00000000000000000000.json"))

add,commitInfo,metaData,protocol
null,"List(0627-190015-worn613, false, WriteSerializable, List(454438568223987), WRITE, List(4, 298347796, 9697750), List(Overwrite, []), 1624825421036, 1007074248881300, silvvavaleria@gmail.com)",null,null
null,null,null,"List(1, 2)"
null,null,"List(1624825328277, List(parquet), aa114718-2b39-4364-b588-d4098d10bebf, List(), {""type"":""struct"",""fields"":[{""name"":""device"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""ecommerce"",""type"":{""type"":""struct"",""fields"":[{""name"":""purchase_revenue_in_usd"",""type"":""double"",""nullable"":true,""metadata"":{}},{""name"":""total_item_quantity"",""type"":""long"",""nullable"":true,""metadata"":{}},{""name"":""unique_items"",""type"":""long"",""nullable"":true,""metadata"":{}}]},""nullable"":true,""metadata"":{}},{""name"":""event_name"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""event_previous_timestamp"",""type"":""long"",""nullable"":true,""metadata"":{}},{""name"":""event_timestamp"",""type"":""long"",""nullable"":true,""metadata"":{}},{""name"":""geo"",""type"":{""type"":""struct"",""fields"":[{""name"":""city"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""state"",""type"":""string"",""nullable"":true,""metadata"":{}}]},""nullable"":true,""metadata"":{}},{""name"":""items"",""type"":{""type"":""array"",""elementType"":{""type"":""struct"",""fields"":[{""name"":""coupon"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""item_id"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""item_name"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""item_revenue_in_usd"",""type"":""double"",""nullable"":true,""metadata"":{}},{""name"":""price_in_usd"",""type"":""double"",""nullable"":true,""metadata"":{}},{""name"":""quantity"",""type"":""long"",""nullable"":true,""metadata"":{}}]},""containsNull"":true},""nullable"":true,""metadata"":{}},{""name"":""traffic_source"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""user_first_touch_timestamp"",""type"":""long"",""nullable"":true,""metadata"":{}},{""name"":""user_id"",""type"":""string"",""nullable"":true,""metadata"":{}}]})",null
"List(true, 1624825414000, part-00000-0c8affaf-83b8-4276-abd9-3b42d0e76aa8-c000.snappy.parquet, 74600524, {""numRecords"":2424522,""minValues"":{""device"":""Android"",""ecommerce"":{""purchase_revenue_in_usd"":53.1,""total_item_quantity"":1,""unique_items"":1},""event_name"":""add_item"",""event_previous_timestamp"":1592205687627986,""event_timestamp"":1592539200194694,""geo"":{""city"":""Abbeville"",""state"":""AK""},""traffic_source"":""direct"",""user_first_touch_timestamp"":1592196947865522,""user_id"":""UA000000102357807""},""maxValues"":{""device"":""macOS"",""ecommerce"":{""purchase_revenue_in_usd"":5289.0,""total_item_quantity"":5,""unique_items"":5},""event_name"":""warranty"",""event_previous_timestamp"":1593879287625340,""event_timestamp"":1593879299370625,""geo"":{""city"":""Zumbrota"",""state"":""WY""},""traffic_source"":""youtube"",""user_first_touch_timestamp"":1593892583883212,""user_id"":""UA000000107499832""},""nullCount"":{""device"":0,""ecommerce"":{""purchase_revenue_in_usd"":2379597,""total_item_quantity"":2379597,""unique_items"":2379597},""event_name"":0,""event_previous_timestamp"":1016253,""event_timestamp"":0,""geo"":{""city"":0,""state"":0},""items"":0,""traffic_source"":0,""user_first_touch_timestamp"":0,""user_id"":0}})",null,null,null
"List(true, 1624825417000, part-00001-2995e88a-2bdb-472b-ad4d-b7d0b567f10b-c000.snappy.parquet, 74596237, {""numRecords"":2424505,""minValues"":{""device"":""Android"",""ecommerce"":{""purchase_revenue_in_usd"":53.1,""total_item_quantity"":1,""unique_items"":1},""event_name"":""add_item"",""event_previous_timestamp"":1592199621810965,""event_timestamp"":1592539202466157,""geo"":{""city"":""Abbeville"",""state"":""AK""},""traffic_source"":""direct"",""user_first_touch_timesta

One key difference between these two transaction logs is the size of the JSON file, this file has 206 rows compared to the previous 7.

To understand why, let's take a look at the `commitInfo` column. We can see that in the `operationParameters` section, `partitionBy` has been filled in by the `state` column. Furthermore, if we look at the add section on row 3, we can see that a new section called `partitionValues` has appeared. As we saw above, Delta stores partitions separately in memory, however, it stores information about these partitions in the same transaction log file.

In [0]:
display(spark.read.json(deltaPath + "/_delta_log/00000000000000000001.json"))

add,commitInfo,metaData,remove
null,"List(0627-190015-worn613, false, WriteSerializable, List(454438568223987), WRITE, List(200, 291969219, 9697750), List(Overwrite, [""state""]), 0, 1624826191367, 1007074248881300, silvvavaleria@gmail.com)",null,null
null,null,"List(1624825328277, List(parquet), aa114718-2b39-4364-b588-d4098d10bebf, List(state), {""type"":""struct"",""fields"":[{""name"":""device"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""ecommerce"",""type"":{""type"":""struct"",""fields"":[{""name"":""purchase_revenue_in_usd"",""type"":""double"",""nullable"":true,""metadata"":{}},{""name"":""total_item_quantity"",""type"":""long"",""nullable"":true,""metadata"":{}},{""name"":""unique_items"",""type"":""long"",""nullable"":true,""metadata"":{}}]},""nullable"":true,""metadata"":{}},{""name"":""event_name"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""event_previous_timestamp"",""type"":""long"",""nullable"":true,""metadata"":{}},{""name"":""event_timestamp"",""type"":""long"",""nullable"":true,""metadata"":{}},{""name"":""geo"",""type"":{""type"":""struct"",""fields"":[{""name"":""city"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""state"",""type"":""string"",""nullable"":true,""metadata"":{}}]},""nullable"":true,""metadata"":{}},{""name"":""items"",""type"":{""type"":""array"",""elementType"":{""type"":""struct"",""fields"":[{""name"":""coupon"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""item_id"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""item_name"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""item_revenue_in_usd"",""type"":""double"",""nullable"":true,""metadata"":{}},{""name"":""price_in_usd"",""type"":""double"",""nullable"":true,""metadata"":{}},{""name"":""quantity"",""type"":""long"",""nullable"":true,""metadata"":{}}]},""containsNull"":true},""nullable"":true,""metadata"":{}},{""name"":""traffic_source"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""user_first_touch_timestamp"",""type"":""long"",""nullable"":true,""metadata"":{}},{""name"":""user_id"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""state"",""type"":""string"",""nullable"":true,""metadata"":{}}]})",null
"List(true, 1624826098000, List(AK), state=AK/part-00000-418ee90a-eba8-48f0-a04c-326a4a1e0d31.c000.snappy.parquet, 58811, {""numRecords"":1655,""minValues"":{""device"":""Android"",""ecommerce"":{""purchase_revenue_in_usd"":59.0,""total_item_quantity"":1,""unique_items"":1},""event_name"":""add_item"",""event_previous_timestamp"":1592323580173801,""event_timestamp"":1592545676942458,""geo"":{""city"":""Akutan"",""state"":""AK""},""traffic_source"":""direct"",""user_first_touch_timestamp"":1592323572704674,""user_id"":""UA000000102702972""},""maxValues"":{""device"":""macOS"",""ecommerce"":{""purchase_revenue_in_usd"":2290.0,""total_item_quantity"":2,""unique_items"":2},""event_name"":""warranty"",""event_previous_timestamp"":1593878868803435,""event_timestamp"":1593879245394745,""geo"":{""city"":""Wasilla"",""state"":""AK""},""traffic_source"":""youtube"",""user_first_touch_timestamp"":1593879245394745,""user_id"":""UA000000107382708""},""nullCount"":{""device"":0,""ecommerce"":{""purchase_revenue_in_usd"":1626,""total_item_quantity"":1626,""unique_items"":1626},""event_name"":0,""event_previous_timestamp"":673,""event_timestamp"":0,""geo"":{""city"":0,""state"":0},""items"":0,""traffic_source"":0,""user_first_touch_timestamp"":0,""user_id"":0}})",null,null,null
"List(true, 1624826101000, List(AL), state=AL/part-00000-a52da13f-c839-4cb0-be47-2a5b86b77f78.c000.snappy.parquet, 1112497, {""numRecords"":36648,""minValues"":{""device"":""Android"",""ecommerce"":{""purchase_revenue_in_usd"":53.1,""total_item_quantity"":1,""unique_items"":1},""event_name"":""add_item"",""event_previous_timestamp"":1592216134896907,""event_timestamp"":1592540545809692,""geo"":{""city"":""Abbeville"

Finally, let's take a look at the files inside one of the state partitions. The files inside corresponds to the partition commit (file 01) in the _delta_log directory.

In [0]:
display(dbutils.fs.ls(deltaPath + "/state=CA/"))

path,name,size
dbfs:/user/silvvavaleria@gmail.com/spark_programming/p/delta-events/state=CA/part-00000-44e8c30f-4570-4b70-93de-984adcfc0142.c000.snappy.parquet,part-00000-44e8c30f-4570-4b70-93de-984adcfc0142.c000.snappy.parquet,12803957
dbfs:/user/silvvavaleria@gmail.com/spark_programming/p/delta-events/state=CA/part-00001-5af89c58-04f6-4fcf-a50e-f83706b6e332.c000.snappy.parquet,part-00001-5af89c58-04f6-4fcf-a50e-f83706b6e332.c000.snappy.parquet,12847990
dbfs:/user/silvvavaleria@gmail.com/spark_programming/p/delta-events/state=CA/part-00002-8cdc16e9-7edd-44e9-a341-c881e6b31bf8.c000.snappy.parquet,part-00002-8cdc16e9-7edd-44e9-a341-c881e6b31bf8.c000.snappy.parquet,12824231
dbfs:/user/silvvavaleria@gmail.com/spark_programming/p/delta-events/state=CA/part-00003-2ff2482f-9a23-4150-a400-e04f023c10cb.c000.snappy.parquet,part-00003-2ff2482f-9a23-4150-a400-e04f023c10cb.c000.snappy.parquet,12834506


### ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Read from your Delta table

In [0]:
df = spark.read.format("delta").load(deltaPath)
display(df)

device,ecommerce,event_name,event_previous_timestamp,event_timestamp,geo,items,traffic_source,user_first_touch_timestamp,user_id,state
Linux,"List(null, null, null)",main,null,1593876884762301,"List(Homer, AK)",List(),facebook,1593876884762301,UA000000107361305,AK
Windows,"List(null, null, null)",mattresses,null,1593879245394745,"List(Ketchikan, AK)",List(),facebook,1593879245394745,UA000000107382708,AK
iOS,"List(null, null, null)",main,null,1593876975161609,"List(Fairbanks, AK)",List(),google,1593876975161609,UA000000107362089,AK
iOS,"List(null, null, null)",warranty,1593878217410311,1593878470430167,"List(Kotzebue, AK)",List(),google,1593878217410311,UA000000107373391,AK
iOS,"List(null, null, null)",add_item,1593877491189853,1593877493241238,"List(Valdez, AK)","List(List(null, M_STAN_F, Standard Full Mattress, 945.0, 945.0, 1), List(null, P_DOWN_S, Standard Down Pillow, 119.0, 119.0, 1))",facebook,1593877109328054,UA000000107363293,AK
iOS,"List(null, null, null)",add_item,1593877109328054,1593877491189853,"List(Valdez, AK)","List(List(null, M_STAN_F, Standard Full Mattress, 945.0, 945.0, 1))",facebook,1593877109328054,UA000000107363293,AK
Chrome OS,"List(null, null, null)",main,null,1593878796707723,"List(Kodiak, AK)",List(),google,1593878796707723,UA000000107378572,AK
Windows,"List(null, null, null)",main,null,1593876547140415,"List(Wasilla, AK)",List(),google,1593876547140415,UA000000107358401,AK
Android,"List(null, null, null)",mattresses,null,1593434019898948,"List(Fairbanks, AK)",List(),facebook,1593434019898948,UA000000106012013,AK
Windows,"List(null, null, null)",main,null,1593433954117798,"List(Kenai, AK)",List(),instagram,1593433954117798,UA000000106011802,AK


### ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Update your Delta Table

Let's filter for rows where the event takes place on a mobile device.

In [0]:
df_update = stateEventsDF.filter(col("device").isin(["Android", "iOS"]))
display(df_update)

device,ecommerce,event_name,event_previous_timestamp,event_timestamp,geo,items,traffic_source,user_first_touch_timestamp,user_id,state
iOS,"List(null, null, null)",mattresses,1593878178791663,1593878809276923,"List(Everett, MA)",List(),facebook,1593877903116176,UA000000107370581,MA
iOS,"List(null, null, null)",main,null,1593877936171803,"List(Mount Pleasant, UT)",List(),direct,1593877936171803,UA000000107370851,UT
Android,"List(null, null, null)",warranty,1593878529774474,1593879213196400,"List(Rancho Santa Margarita, CA)",List(),instagram,1593878529774474,UA000000107376205,CA
iOS,"List(null, null, null)",original,1593878068949001,1593878170903989,"List(Longview, WA)",List(),google,1593877826716812,UA000000107369909,WA
Android,"List(null, null, null)",down,1593879057792999,1593879125815755,"List(Jackson, MO)",List(),facebook,1593879057792999,UA000000107380961,MO
iOS,"List(null, null, null)",mattresses,null,1593876687337581,"List(Warwick, RI)",List(),google,1593876687337581,UA000000107359573,RI
iOS,"List(null, null, null)",original,1593877781854634,1593877788141768,"List(Dunwoody, GA)",List(),google,1593877781854634,UA000000107369512,GA
iOS,"List(null, null, null)",main,1593877445670953,1593877497207417,"List(Rochester, MN)",List(),facebook,1593877300577217,UA000000107365065,MN
Android,"List(null, null, null)",main,null,1593877271099453,"List(Raleigh, NC)",List(),direct,1593877271099453,UA000000107364813,NC
iOS,"List(null, null, null)",reviews,1593878353149193,1593878356880855,"List(Los Angeles, CA)",List(),facebook,1593878353149193,UA000000107374663,CA


In [0]:
df_update.write.format("delta").mode("overwrite").save(deltaPath)

In [0]:
df = spark.read.format("delta").load(deltaPath)
display(df)

device,ecommerce,event_name,event_previous_timestamp,event_timestamp,geo,items,traffic_source,user_first_touch_timestamp,user_id,state
iOS,"List(null, null, null)",careers,1593878436984839,1593878541308171,"List(Birmingham, AL)",List(),google,1593877715595422,UA000000107368898,AL
Android,"List(null, null, null)",main,null,1593878511225558,"List(Huntsville, AL)",List(),youtube,1593878511225558,UA000000107376048,AL
Android,"List(null, null, null)",main,null,1593877091265157,"List(Daphne, AL)",List(),instagram,1593877091265157,UA000000107363133,AL
iOS,"List(null, null, null)",main,null,1593877001288343,"List(Tuscumbia, AL)",List(),facebook,1593877001288343,UA000000107362306,AL
iOS,"List(null, null, null)",main,null,1593878851818181,"List(Montgomery, AL)",List(),instagram,1593878851818181,UA000000107379093,AL
Android,"List(null, null, null)",add_item,1593878817421190,1593878902626089,"List(Montgomery, AL)","List(List(null, M_STAN_Q, Standard Queen Mattress, 1045.0, 1045.0, 1))",google,1593878817421190,UA000000107378772,AL
iOS,"List(null, null, null)",mattresses,null,1593877302755243,"List(Mobile, AL)",List(),direct,1593877302755243,UA000000107365087,AL
Android,"List(null, null, null)",main,null,1593877483896790,"List(Birmingham, AL)",List(),facebook,1593877483896790,UA000000107366773,AL
iOS,"List(null, null, null)",original,1593877514211341,1593878119218617,"List(Auburn, AL)",List(),email,1593877514211341,UA000000107367062,AL
iOS,"List(null, null, null)",main,1593879013515249,1593879064666694,"List(Hoover, AL)",List(),instagram,1593878493879222,UA000000107375879,AL


Let's look at the files in the California partition post-update. Remember, the different files in this directory are snapshots of your DataFrame corresponding to different commits.

In [0]:
display(dbutils.fs.ls(deltaPath + "/state=CA/"))

path,name,size
dbfs:/user/silvvavaleria@gmail.com/spark_programming/p/delta-events/state=CA/part-00000-44e8c30f-4570-4b70-93de-984adcfc0142.c000.snappy.parquet,part-00000-44e8c30f-4570-4b70-93de-984adcfc0142.c000.snappy.parquet,12803957
dbfs:/user/silvvavaleria@gmail.com/spark_programming/p/delta-events/state=CA/part-00000-6eb9b0c5-08aa-49dd-9c4a-cfb76cc13d00.c000.snappy.parquet,part-00000-6eb9b0c5-08aa-49dd-9c4a-cfb76cc13d00.c000.snappy.parquet,5738470
dbfs:/user/silvvavaleria@gmail.com/spark_programming/p/delta-events/state=CA/part-00001-157b7f4f-079a-4dd3-8a9b-0c51702b0294.c000.snappy.parquet,part-00001-157b7f4f-079a-4dd3-8a9b-0c51702b0294.c000.snappy.parquet,5757076
dbfs:/user/silvvavaleria@gmail.com/spark_programming/p/delta-events/state=CA/part-00001-5af89c58-04f6-4fcf-a50e-f83706b6e332.c000.snappy.parquet,part-00001-5af89c58-04f6-4fcf-a50e-f83706b6e332.c000.snappy.parquet,12847990
dbfs:/user/silvvavaleria@gmail.com/spark_programming/p/delta-events/state=CA/part-00002-8cdc16e9-7edd-44e9-a341-c881e6b31bf8.c000.snappy.parquet,part-00002-8cdc16e9-7edd-44e9-a341-c881e6b31bf8.c000.snappy.parquet,12824231
dbfs:/user/silvvavaleria@gmail.com/spark_programming/p/delta-events/state=CA/part-00002-bd224c7b-8ed8-4d37-84fc-b56e304f0175.c000.snappy.parquet,part-00002-bd224c7b-8ed8-4d37-84fc-b56e304f0175.c000.snappy.parquet,5736201
dbfs:/user/silvvavaleria@gmail.com/spark_programming/p/delta-events/state=CA/part-00003-2ff2482f-9a23-4150-a400-e04f023c10cb.c000.snappy.parquet,part-00003-2ff2482f-9a23-4150-a400-e04f023c10cb.c000.snappy.parquet,12834506
dbfs:/user/silvvavaleria@gmail.com/spark_programming/p/delta-events/state=CA/part-00003-c49af2cc-e654-4490-afdb-cd04a9923f99.c000.snappy.parquet,part-00003-c49af2cc-e654-4490-afdb-cd04a9923f99.c000.snappy.parquet,5731766


### ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Access previous versions of table using Time  Travel

Oops, it turns out we actually we need the entire dataset! You can access a previous version of your Delta Table using Time Travel. Use the following two cells to access your version history. Delta Lake will keep a 30 day version history by default, but if necessary, Delta can store a version history for longer.

In [0]:
spark.sql("DROP TABLE IF EXISTS train_delta")
spark.sql(f"CREATE TABLE train_delta USING DELTA LOCATION '{deltaPath}'")

Out[17]: DataFrame[]

In [0]:
%sql
DESCRIBE HISTORY train_delta

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata
2,2021-06-27T20:39:10.000+0000,1007074248881300,silvvavaleria@gmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(454438568223987),0627-190015-worn613,1,WriteSerializable,false,"Map(numFiles -> 200, numOutputBytes -> 132262981, numOutputRows -> 4361535)",null
1,2021-06-27T20:36:32.000+0000,1007074248881300,silvvavaleria@gmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [""state""])",null,List(454438568223987),0627-190015-worn613,0,WriteSerializable,false,"Map(numFiles -> 200, numOutputBytes -> 291969219, numOutputRows -> 9697750)",null
0,2021-06-27T20:23:42.000+0000,1007074248881300,silvvavaleria@gmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(454438568223987),0627-190015-worn613,null,WriteSerializable,false,"Map(numFiles -> 4, numOutputBytes -> 298347796, numOutputRows -> 9697750)",null


Using the `versionAsOf` option allows you to easily access previous versions of our Delta Table.

In [0]:
df = spark.read.format("delta").option("versionAsOf", 0).load(deltaPath)
display(df)

device,ecommerce,event_name,event_previous_timestamp,event_timestamp,geo,items,traffic_source,user_first_touch_timestamp,user_id
macOS,"List(null, null, null)",warranty,1593878899217692,1593878946592107,"List(Montrose, MI)",List(),google,1593878899217692,UA000000107379500
Windows,"List(null, null, null)",press,1593876662175340,1593877011756535,"List(Northampton, MA)",List(),google,1593876662175340,UA000000107359357
macOS,"List(null, null, null)",add_item,1593878792892652,1593878815459100,"List(Salinas, CA)","List(List(null, M_STAN_T, Standard Twin Mattress, 595.0, 595.0, 1))",youtube,1593878455472030,UA000000107375547
iOS,"List(null, null, null)",mattresses,1593878178791663,1593878809276923,"List(Everett, MA)",List(),facebook,1593877903116176,UA000000107370581
Windows,"List(null, null, null)",mattresses,null,1593878628143633,"List(Cottage Grove, MN)",List(),google,1593878628143633,UA000000107377108
Windows,"List(null, null, null)",main,null,1593878634344194,"List(Medina, MN)",List(),youtube,1593878634344194,UA000000107377161
iOS,"List(null, null, null)",main,null,1593877936171803,"List(Mount Pleasant, UT)",List(),direct,1593877936171803,UA000000107370851
macOS,"List(null, null, null)",main,null,1593876843215329,"List(Piedmont, AL)",List(),instagram,1593876843215329,UA000000107360961
Android,"List(null, null, null)",warranty,1593878529774474,1593879213196400,"List(Rancho Santa Margarita, CA)",List(),instagram,1593878529774474,UA000000107376205
Windows,"List(null, null, null)",main,null,1593876713246514,"List(Elyria, OH)",List(),facebook,1593876713246514,UA000000107359805


You can also access older versions using a timestamp.

Replace the timestamp string with the information from your version history. Note that you can use a date without the time information if necessary.

In [0]:
# TODO
timeStampString = "2021-06-27T20:39:10.000+0000"
df = spark.read.format("delta").option("timestampAsOf", timeStampString).load(deltaPath)
display(df)

device,ecommerce,event_name,event_previous_timestamp,event_timestamp,geo,items,traffic_source,user_first_touch_timestamp,user_id,state
iOS,"List(null, null, null)",careers,1593878436984839,1593878541308171,"List(Birmingham, AL)",List(),google,1593877715595422,UA000000107368898,AL
Android,"List(null, null, null)",main,null,1593878511225558,"List(Huntsville, AL)",List(),youtube,1593878511225558,UA000000107376048,AL
Android,"List(null, null, null)",main,null,1593877091265157,"List(Daphne, AL)",List(),instagram,1593877091265157,UA000000107363133,AL
iOS,"List(null, null, null)",main,null,1593877001288343,"List(Tuscumbia, AL)",List(),facebook,1593877001288343,UA000000107362306,AL
iOS,"List(null, null, null)",main,null,1593878851818181,"List(Montgomery, AL)",List(),instagram,1593878851818181,UA000000107379093,AL
Android,"List(null, null, null)",add_item,1593878817421190,1593878902626089,"List(Montgomery, AL)","List(List(null, M_STAN_Q, Standard Queen Mattress, 1045.0, 1045.0, 1))",google,1593878817421190,UA000000107378772,AL
iOS,"List(null, null, null)",mattresses,null,1593877302755243,"List(Mobile, AL)",List(),direct,1593877302755243,UA000000107365087,AL
Android,"List(null, null, null)",main,null,1593877483896790,"List(Birmingham, AL)",List(),facebook,1593877483896790,UA000000107366773,AL
iOS,"List(null, null, null)",original,1593877514211341,1593878119218617,"List(Auburn, AL)",List(),email,1593877514211341,UA000000107367062,AL
iOS,"List(null, null, null)",main,1593879013515249,1593879064666694,"List(Hoover, AL)",List(),instagram,1593878493879222,UA000000107375879,AL


### ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Vacuum

Now that we're happy with our Delta Table, we can clean up our directory using `VACUUM`. Vacuum accepts a retention period in hours as an input.

It looks like our code doesn't run! By default, to prevent accidentally vacuuming recent commits, Delta Lake will not let users vacuum a period under 7 days or 168 hours. Once vacuumed, you cannot return to a prior commit through time travel, only your most recent Delta Table will be saved.

In [0]:
from delta.tables import *

deltaTable = DeltaTable.forPath(spark, deltaPath)
deltaTable.vacuum()

Out[24]: DataFrame[]

We can workaround this by setting a spark configuration that will bypass the default retention period check.

In [0]:
from delta.tables import *

spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", "false")
deltaTable = DeltaTable.forPath(spark, deltaPath)
deltaTable.vacuum(0)

Out[25]: DataFrame[]

Let's take a look at our Delta Table files now. After vacuuming, the directory only holds the partition of our most recent Delta Table commit.

In [0]:
display(dbutils.fs.ls(deltaPath + "/state=CA/"))

path,name,size
dbfs:/user/silvvavaleria@gmail.com/spark_programming/p/delta-events/state=CA/part-00000-6eb9b0c5-08aa-49dd-9c4a-cfb76cc13d00.c000.snappy.parquet,part-00000-6eb9b0c5-08aa-49dd-9c4a-cfb76cc13d00.c000.snappy.parquet,5738470
dbfs:/user/silvvavaleria@gmail.com/spark_programming/p/delta-events/state=CA/part-00001-157b7f4f-079a-4dd3-8a9b-0c51702b0294.c000.snappy.parquet,part-00001-157b7f4f-079a-4dd3-8a9b-0c51702b0294.c000.snappy.parquet,5757076
dbfs:/user/silvvavaleria@gmail.com/spark_programming/p/delta-events/state=CA/part-00002-bd224c7b-8ed8-4d37-84fc-b56e304f0175.c000.snappy.parquet,part-00002-bd224c7b-8ed8-4d37-84fc-b56e304f0175.c000.snappy.parquet,5736201
dbfs:/user/silvvavaleria@gmail.com/spark_programming/p/delta-events/state=CA/part-00003-c49af2cc-e654-4490-afdb-cd04a9923f99.c000.snappy.parquet,part-00003-c49af2cc-e654-4490-afdb-cd04a9923f99.c000.snappy.parquet,5731766


Since vacuuming deletes files referenced by the Delta Table, we can no longer access past versions. The code below should throw an error.

In [0]:
df = spark.read.format("delta").option("versionAsOf", 1).load(deltaPath)
display(df)

org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 213.0 failed 1 times, most recent failure: Lost task 0.0 in stage 213.0 (TID 1369) (ip-10-172-209-217.us-west-2.compute.internal executor driver): com.databricks.sql.io.FileReadException: Error while reading file dbfs:REDACTED_LOCAL_PART@gmail.com/spark_programming/p/delta-events/state=AK/part-00000-418ee90a-eba8-48f0-a04c-326a4a1e0d31.c000.snappy.parquet. A file referenced in the transaction log cannot be found. This occurs when data has been manually deleted from the file system rather than using the table `DELETE` statement. For more information, see https://docs.databricks.com/delta/delta-intro.html#frequently-asked-questions
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anon$2.logFileNameAndThrow(FileScanRDD.scala:389)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anon$2.getNext(FileScanRDD.scala:347)
	at org.apache.spark.util.NextIterator.hasNext(NextIterator.scala:73)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.$anonfun$prepareNextFile$1(FileScanRDD.scala:536)
	at scala.concurrent.Future$.$anonfun$apply$1(Future.scala:659)
	at scala.util.Success.$anonfun$map$1(Try.scala:255)
	at scala.util.Success.map(Try.scala:213)
	at scala.concurrent.Future.$anonfun$map$1(Future.scala:292)
	at scala.concurrent.impl.Promise.liftedTree1$1(Promise.scala:33)
	at scala.concurrent.impl.Promise.$anonfun$transform$1(Promise.scala:33)
	at scala.concurrent.impl.CallbackRunnable.run(Promise.scala:64)
	at org.apache.spark.util.threads.SparkThreadLocalCapturingRunnable.$anonfun$run$1(SparkThreadLocalForwardingThreadPoolExecutor.scala:104)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.util.threads.SparkThreadLocalCapturingHelper.runWithCaptured(SparkThreadLocalForwardingThreadPoolExecutor.scala:68)
	at org.apache.spark.util.threads.SparkThreadLocalCapturingHelper.runWithCaptured$(SparkThreadLocalForwardingThreadPoolExecutor.scala:54)
	at org.apache.spark.util.threads.SparkThreadLocalCapturingRunnable.runWithCaptured(SparkThreadLocalForwardingThreadPoolExecutor.scala:101)
	at org.apache.spark.util.threads.SparkThreadLocalCapturingRunnable.run(SparkThreadLocalForwardingThreadPoolExecutor.scala:104)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.io.FileNotFoundException: /user/silvvavaleria@gmail.com/spark_programming/p/delta-events/state=AK/part-00000-418ee90a-eba8-48f0-a04c-326a4a1e0d31.c000.snappy.parquet
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:120)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:68)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV1.getFileStatus(DatabricksFileSystemV1.scala:270)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystem.getFileStatus(DatabricksFileSystem.scala:201)
	at com.databricks.spark.metrics.FileSystemWithMetrics.getFileStatus(FileSystemWithMetrics.scala:303)
	at com.databricks.sql.io.parquet.CachingParquetFileReader.readFooter(CachingParquetFileReader.java:482)
	at org.apache.spark.sql.execution.datasources.parquet.SpecificParquetRecordReaderBase.prepare(SpecificParquetRecordReaderBase.java:137)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$$anon$2.apply(ParquetFileFormat.scala:466)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$$anon$2.apply(ParquetFileFormat.scala:340)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anon$2.getNext(FileScanRDD.scala:331)
	... 18 more

Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2766)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGSchedu

### Clean up classroom

In [0]:
%run ./Includes/Classroom-Cleanup
